In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import strftime
import numpy as np

import scrape_url_list as s
import acquire as a
import prepare as p

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

---

In [2]:
links = s.get_cached_links()

In [3]:
links = links[:-2]

---

In [4]:
link_list = []
for link in links:
    response = requests.get(link, headers={"user-agent": "Codeup DS"})
    link_list.append({'link' : link, 'status_code' : response.status_code})

In [5]:
df = pd.DataFrame(link_list)

In [6]:
df.head()

,link,status_code
0,https://github.com/josephmisiti/awesome-machin...,200
1,https://github.com/microsoft/ML-For-Beginners,200
2,https://github.com/bfortuner/ml-glossary,200
3,https://github.com/trekhleb/homemade-machine-l...,200
4,https://github.com/dangkhoasdc/awesome-ai-resi...,200


In [7]:
df[df.status_code != 200]

,link,status_code


#### Takeaways from acquire:
- The urls were fine when making requests, all returned status code of 200
- But when I tried to use Zach's acquire module to get the needed content, 5 links returned `404` errors
    - Still not sure exaxtly why, but must have to do with the scopes associated with my personal access token and auther permissions or someting
    - Here are the ones that didn't work:
        - `aikorea/awesome-rl/`
        - `JoseDeFreitas/awesome-youtubers#machine-learning`
        - `Developer-Y/cs-video-courses#artificial-intelligence`
        - `Vedenin/useful-java-links#ii-databases-search-engines-big-data-and-machine-learning`
        - `alirezadir/machine-learning-interview-enlightener/blob/main/README.md`

In [9]:
df = pd.read_json('data.json')

In [10]:
df.head()

,repo,language,readme_contents
0,codingXiaxw/CustomerManagement,Java,## JavaWeb项目之客户管理系统 \n\n## 项目介绍\n详细讲解请点击这里前往我的...
1,DanWahlin/CustomerManager,JavaScript,Customer Manager with AngularJS (with custom r...
2,Tophold/FinancialCustomerView,Java,## FinancialCustomerView\n* 本项目会对金融交易软件中存在的各种V...
3,DanWahlin/CustomerManagerStandard,JavaScript,Customer Manager with AngularJS\r\n===========...
4,spring-cloud-samples/customers-stores,Java,# REST based micro-services sample\n\n- Three ...


In [11]:
df.shape

(194, 3)

In [12]:
df.language.value_counts(dropna=False)

PHP                 41
Jupyter Notebook    32
JavaScript          25
Java                23
Python              20
C#                  12
NaN                  7
TypeScript           4
HTML                 4
R                    4
Ruby                 3
Go                   3
Vue                  3
CSS                  2
HCL                  2
Dart                 2
PowerShell           2
Objective-C          1
Shell                1
ABAP                 1
Scala                1
Swift                1
Name: language, dtype: int64

In [70]:
# df.loc[0 ,'readme_contents']

---

Get some new links

In [13]:
response = requests.get('https://github.com/trending', headers={"user-agent": "Codeup DS"})
soup = BeautifulSoup(response.text)

In [16]:
len(soup.select('.Box-row'))

25

In [23]:
[link.attrs['href'] for link in soup.select_one('.Box').select('a')]

['', '', '', '']

In [24]:
# soup.select_one('.Box').select('a')

In [48]:
soup.select('.Box-row')[0].select_one('.h3.lh-condensed').a.attrs['href']

'/github/copilot.vim'

In [49]:
boxes = soup.select('.Box-row')

In [51]:
len(boxes)

25

In [52]:
link_list = []
for box in boxes:
    link = box.select_one('.h3.lh-condensed').a.attrs['href']
    link_list.append(link)

In [53]:
link_list

['/github/copilot.vim',
 '/schollz/croc',
 '/arco-design/arco-design-vue',
 '/iterativv/NostalgiaForInfinity',
 '/WorkerLivesMatter/WorkingTime',
 '/ehn-dcc-development/hcert-spec',
 '/kon9chunkit/GitHub-Chinese-Top-Charts',
 '/vercel/next.js',
 '/github/copilot-docs',
 '/2point21/lba1-classic',
 '/shengcaishizhan/kkndme_tianya',
 '/2point21/lba2-classic',
 '/DouyinFE/semi-design',
 '/Leaflet/Leaflet',
 '/modern-js-dev/modern.js',
 '/devMEremenko/XcodeBenchmark',
 '/waldo-vision/optical-flow-demo',
 '/angular/components',
 '/projectdiscovery/nuclei-templates',
 '/onevcat/Kingfisher',
 '/czj2369/jd_tb_auto',
 '/mltframework/shotcut',
 '/arco-design/arco-design',
 '/jumpserver/jumpserver',
 '/tailwindlabs/headlessui']

In [54]:
# there are only 25 links here that are classified as trending

---

Most forked

In [51]:
url = 'https://github.com/search?o=desc&p=1&q=stars%3A%3E1&s=forks&type=Repositories'

In [52]:
# get first page
response = requests.get(url, headers={"user-agent": "Codeup DS"})
soup = BeautifulSoup(response.text)

In [53]:
len(soup.select('.repo-list-item'))

10

In [61]:
cards = soup.select('.repo-list-item')

In [62]:
len(cards)

10

In [68]:
cards[0].select_one('.v-align-middle').attrs['href']

'/jtleek/datasharing'

In [69]:
link_list = []
for card in cards:
    link = card.select_one('.v-align-middle').attrs['href']
    link_list.append(link)

In [70]:
link_list

['/jtleek/datasharing',
 '/rdpeng/ProgrammingAssignment2',
 '/octocat/Spoon-Knife',
 '/tensorflow/tensorflow',
 '/twbs/bootstrap',
 '/github/gitignore',
 '/Pierian-Data/Complete-Python-3-Bootcamp',
 '/nightscout/cgm-remote-monitor',
 '/rdpeng/ExData_Plotting1',
 '/jwasham/coding-interview-university']

In [54]:
# get 10 pages
url = 'https://github.com/search?o=desc&p={}&q=stars%3A%3E1&s=forks&type=Repositories'

In [55]:
url = url.format(1)

In [56]:
url

'https://github.com/search?o=desc&p=1&q=stars%3A%3E1&s=forks&type=Repositories'

In [57]:
response = requests.get(url, headers={"user-agent": "Codeup DS"})

In [58]:
response

<Response [200]>

In [30]:
soup = BeautifulSoup(response.text)

In [31]:
cards = soup.select('.repo-list-item')

In [33]:
len(cards)

10

In [9]:
# link_list = []
# for i in range(1,10):
#     url = 'https://github.com/search?o=desc&p={}&q=stars%3A%3E1&s=forks&type=Repositories'
#     url = url.format(i)
#     response = requests.get(url, headers={"user-agent": "Codeup DS"})
#     soup = BeautifulSoup(response.text)
#     cards = soup.select('.repo-list-item')
#     for card in cards:
#         link = card.select_one('.v-align-middle').attrs['href']
#         link_list.append(link)
    

In [65]:
from time import sleep
cards_list = []
link_list = []
for i in range(1,51):
    url = 'https://github.com/search?o=desc&p={}&q=stars%3A%3E1&s=forks&type=Repositories'
    url = url.format(i)
    response = requests.get(url, headers={"user-agent": "Codeup"})
    print(response.status_code)
    soup = BeautifulSoup(response.text)
    cards = soup.select('.repo-list-item')
    cards_list.append(cards)
    #print(i)
    #print(url)
    #print(len(cards))
    #print(len(cards_list))
    for card in cards:
        link = card.select_one('.v-align-middle').attrs['href']
        link_list.append(link)
    sleep(30)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [66]:
len(link_list)

500

In [69]:
link_list[0][1:]

'jtleek/datasharing'

In [70]:
[link[1:] for link in link_list]

['jtleek/datasharing',
 'rdpeng/ProgrammingAssignment2',
 'octocat/Spoon-Knife',
 'tensorflow/tensorflow',
 'SmartThingsCommunity/SmartThingsPublic',
 'twbs/bootstrap',
 'github/gitignore',
 'Pierian-Data/Complete-Python-3-Bootcamp',
 'nightscout/cgm-remote-monitor',
 'rdpeng/ExData_Plotting1',
 'jwasham/coding-interview-university',
 'opencv/opencv',
 'EbookFoundation/free-programming-books',
 'CyC2018/CS-Notes',
 'tensorflow/models',
 'eugenp/tutorials',
 'jackfrued/Python-100-Days',
 'firstcontributions/first-contributions',
 'torvalds/linux',
 'Snailclimb/JavaGuide',
 'rdpeng/RepData_PeerAssessment1',
 'facebook/react',
 'spring-projects/spring-boot',
 'jlord/patchwork',
 'TheAlgorithms/Python',
 'barryclark/jekyll-now',
 'spring-projects/spring-framework',
 'DataScienceSpecialization/courses',
 'ant-design/ant-design',
 'vuejs/vue',
 'github/docs',
 'getify/You-Dont-Know-JS',
 'angular/angular.js',
 'donnemartin/system-design-primer',
 'freeCodeCamp/freeCodeCamp',
 'DefinitelyType

In [67]:
link_list

['/jtleek/datasharing',
 '/rdpeng/ProgrammingAssignment2',
 '/octocat/Spoon-Knife',
 '/tensorflow/tensorflow',
 '/SmartThingsCommunity/SmartThingsPublic',
 '/twbs/bootstrap',
 '/github/gitignore',
 '/Pierian-Data/Complete-Python-3-Bootcamp',
 '/nightscout/cgm-remote-monitor',
 '/rdpeng/ExData_Plotting1',
 '/jwasham/coding-interview-university',
 '/opencv/opencv',
 '/EbookFoundation/free-programming-books',
 '/CyC2018/CS-Notes',
 '/tensorflow/models',
 '/eugenp/tutorials',
 '/jackfrued/Python-100-Days',
 '/firstcontributions/first-contributions',
 '/torvalds/linux',
 '/Snailclimb/JavaGuide',
 '/rdpeng/RepData_PeerAssessment1',
 '/facebook/react',
 '/spring-projects/spring-boot',
 '/jlord/patchwork',
 '/TheAlgorithms/Python',
 '/barryclark/jekyll-now',
 '/spring-projects/spring-framework',
 '/DataScienceSpecialization/courses',
 '/ant-design/ant-design',
 '/vuejs/vue',
 '/github/docs',
 '/getify/You-Dont-Know-JS',
 '/angular/angular.js',
 '/donnemartin/system-design-primer',
 '/freeCodeC

---

In [2]:
df = pd.read_json('data.json')

In [3]:
df.head()

,repo,language,readme_contents
0,jtleek/datasharing,None,How to share data with a statistician\n=======...
1,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...
2,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...
3,tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www...."
4,SmartThingsCommunity/SmartThingsPublic,Groovy,# SmartThings Public GitHub Repo\n\nAn officia...


In [4]:
df.shape

(500, 3)

In [5]:
df.language.value_counts(dropna=False)

JavaScript          100
Java                 77
NaN                  60
Python               54
C++                  30
Jupyter Notebook     26
TypeScript           26
Go                   19
HTML                 16
C                    15
PHP                  12
Ruby                 11
Shell                 8
CSS                   5
Kotlin                5
C#                    4
Swift                 4
Vue                   3
Rascal                3
Nix                   2
Dart                  2
Objective-C           2
R                     2
Scala                 2
Assembly              1
Less                  1
Groovy                1
Vim script            1
PowerShell            1
Dockerfile            1
Rust                  1
TeX                   1
Rich Text Format      1
Nunjucks              1
ApacheConf            1
SCSS                  1
Name: language, dtype: int64

In [7]:
df = p.prep_nlp(df, original_text_col='readme_contents')

In [8]:
df.head()

,repo,language,readme_contents,clean,stemmed,lemmatized
0,jtleek/datasharing,None,How to share data with a statistician\n=======...,share data statistician guide anyone needs sha...,share data statistician guid anyon need share ...,share data statistician guide anyone need shar...
1,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction second programming assignment req...,introduct second program assign requir write r...,introduction second programming assignment req...
2,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello repository meant provide example fo...,well hello repositori meant provid exampl fork...,well hello repository meant provide example fo...
3,tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www....",div aligncenter img srchttpswwwtensorfloworgim...,div aligncent img srchttpswwwtensorfloworgimag...,div aligncenter img srchttpswwwtensorfloworgim...
4,SmartThingsCommunity/SmartThingsPublic,Groovy,# SmartThings Public GitHub Repo\n\nAn officia...,smartthings public github repo official list s...,smartth public github repo offici list smartap...,smartthings public github repo official list s...


In [9]:
df.shape

(500, 6)